In [1]:
import cv2
import itertools as it
import numpy as np
import pickle
import random

from utilities import *

In [2]:
top, left, bottom, right = 609, 381, 625, 501
width, height = right - left, bottom - top

file_titles = [
    'ancient_apparition',
    'anti-mage',
    'broodmother',
    'centaur_warrunner',
    'clinkz',
    'io',
    'juggernaut',
    'keeper_of_the_light',
    "nature's_prophet",
    'nyx_assassin',
    'outworld_devourer',
    'queen_of_pain',
]
s = ''.join(c for c, _ in it.groupby(sorted(it.chain.from_iterable(file_titles))))
print(len(file_titles), s, len(s))

12 '-_abcdefghijklmnopqrstuvwxyz 29


In [3]:
def fn(file_title):
    with open(fr"D:\Dota 2\Heroes\Pickles\{file_title}.pickle", 'rb') as fin:
        return tuple(pickle_iter(fin))
data = {s: fn(s) for s in file_titles}
show_and_wait(np.vstack(list(data.items())[0][1][:33]))
cv2.destroyAllWindows()
image = list(data.items())[0][1][0]
print([len(v) for _, v in data.items()])
len(data), image.shape, image.dtype, image.min(), image.max()

[23400, 26970, 27030, 25110, 27030, 28920, 26730, 13020, 25500, 27000, 26430, 25830]


(12, (16, 120, 3), dtype('uint8'), 0, 255)

In [5]:
# Color data adds no value to this methodology.
# Transform all data into greyscale.
data = {k: tuple(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in v) for k, v in data.items()}

In [4]:
range_args = (120, 211, 15)
print(*range(*range_args))
known_thresholds = list(range(*range_args))
known_thresholds[-1] = -1

120 135 150 165 180 195 210


In [7]:
# Based on this experiment, it appears 165 is a good threshold for most of the data but, for "outworld devourer",
# this needs to be no more than 135.  Using an adaptive thresholding mechanism ensures a suitable threshold for all data.
# The same threshold excludes more background pixels in greyscale than in color.
def fn(k, v):
    def fn(index, threshold):
        image = v[index].copy()
        if threshold > 0:
            image[image < threshold] = 0
        else:
            threshold, image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
            computed_thresholds.add(threshold)
        return image
    indices = sorted(random.sample(range(len(v)), k=99))
    computed_thresholds = set()
    show_and_wait(np.hstack([np.vstack([fn(i, n) for i in indices]) for n in known_thresholds]))
    print(k, min(computed_thresholds), max(computed_thresholds))
[fn(k, v) for k, v in data.items()]
cv2.destroyAllWindows()

ancient_apparition 118.0 167.0
anti-mage 96.0 177.0
broodmother 118.0 178.0
centaur_warrunner 109.0 179.0
clinkz 91.0 178.0
io 54.0 153.0
juggernaut 98.0 181.0
keeper_of_the_light 117.0 184.0
nature's_prophet 118.0 175.0
nyx_assassin 118.0 173.0
outworld_devourer 97.0 163.0
queen_of_pain 111.0 178.0


In [ ]:
cv2.namedWindow('tesst', cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_EXPANDED)
limits = {}
for k, v in data.items():
    while True:
        image = random.choice(v)
        _, image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
        if show_and_wait(image) == 'g':
            break
    left_limit, right_limit = 0, image.shape[1]
    while True:
        ch = show_and_wait(np.hstack([np.ones([image.shape[0] - 7, left_limit], dtype=image.dtype) * 128,
                                      image[4:-3, left_limit:right_limit],
                                      np.ones([image.shape[0] - 7, image.shape[1] - right_limit], dtype=image.dtype) * 128]))
        if ch == 's':
            left_limit -= 1
        elif ch == 'f':
            left_limit += 1
        elif ch == 'j':
            right_limit -= 1
        elif ch == 'l':
            right_limit += 1
        elif ch in [' ', 'q']:
            break
        else:
            print(ch, ord(ch))
    if ch == 'q':
        break
    limits[k] = (left_limit, right_limit)
cv2.destroyAllWindows()

In [31]:
# These are the limits created above.
limits = {'ancient_apparition': (14, 117),
          'anti-mage': (33, 98),
          'broodmother': (22, 110),
          'centaur_warrunner': (13, 118),
          'clinkz': (44, 87),
          'io': (58, 74),
          'juggernaut': (27, 104),
          'keeper_of_the_light': (14, 118),
          "nature's_prophet": (12, 120),
          'nyx_assassin': (25, 106),
          'outworld_devourer': (14, 117),
          'queen_of_pain': (21, 110)}

In [8]:
def fn(file_title):
    # Select image.
    while True:
        image = random.choice(data[file_title])
        if show_and_wait(image) == 'g':
            break

    # Compute threshold
    _, image = cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)
    horizontal_offset = 14
    show_and_wait(image[4:-3, 14:-3])

    # Determine possible horizontal extents for the letters in the image.
    def fn(image):
        l = []
        for x in range(image.shape[1] - 1):
            if not image[:, x].any() and image[:, x + 1].any():
                l.append([x + horizontal_offset])
            elif len(l) and image[:, x].any() and not image[:, x + 1].any():
                l[-1].append(x + 1 + horizontal_offset)
        return tuple(map(tuple, l))
    g = (cv2.threshold(image, 0, 0, cv2.THRESH_TOZERO | cv2.THRESH_OTSU)[1] for image in data["outworld_devourer"][:1111])
    g = (fn(image[4:-4, horizontal_offset:-3]) for image in g)
    s = {t for t in g if len(t) == 16 and all(len(t) == 2 for t in t)}
    g = map(set, zip(*s))
    horizontal_extents = tuple((min(a for a, _ in s), max(b for _, b in s), s) for s in g)
    print(*horizontal_extents, sep='\n')

cv2.namedWindow('tesst', cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_EXPANDED)
indicators_image = np.zeros([2, reference_image.shape[1]], dtype=reference_image.dtype)
selected_extents = []
for horizontal_extent in horizontal_extents:
    left_offset = right_offset = 0
    while True:
        image = np.vstack([reference_image, indicators_image])
        for selected_extent in selected_extents:
            image[-1, selected_extent[0]:selected_extent[1]] = 128
        image[-2, horizontal_extent[0] + left_offset:horizontal_extent[1] + right_offset] = 255
        ch = show_and_wait(image)
        if ch == 's':
            left_offset -= 1
        elif ch == 'f':
            left_offset += 1
        elif ch == 'j':
            right_offset -= 1
        elif ch == 'l':
            right_offset += 1
        elif ch in [' ', 'q']:
            break
        else:
            print(ch, ord(ch))
    if ch == 'q':
        break
    selected_extents.append([horizontal_extent[0] + left_offset, horizontal_extent[1] + right_offset])
cv2.destroyAllWindows()
print(*selected_extents)